## Libraries

In [1]:
import pickle as pk
import pandas as pd
from datetime import datetime

## Reading the invoices_new.pkl file

In [2]:
with open('invoices_new.pkl', 'rb') as file:
    invoices = pk.load(file)

In [3]:
print(invoices[0])

{'id': '365371O', 'created_on': '2019-05-27', 'items': [{'item': {'id': 142446, 'name': 'ii_142446', 'unit_price': 182, 'type': 1}, 'quantity': 6}, {'item': {'id': 154019, 'name': 'ii_154019', 'unit_price': 176, 'type': 1}, 'quantity': 1}]}


## Reading the expired_invoices.txt file

#### 1. checking the content of the txt file

In [4]:
with open('expired_invoices.txt', 'r') as file:
    content = file.read()
    
print(content)

305869, 377307, 391273, 385290, 331902, 397723, 343695, 340601, 347510, 325156, 379387, 348894, 322229, 326649, 377960, 325063, 369378, 319405, 383681, 367288, 356552, 394428, 381476, 381457, 349879, 343254, 366751, 330931, 363263, 326452, 379687, 356532, 383235, 313012, 368913, 352442, 315960, 351096, 331193, 392657, 352391, 389528, 338547, 379961, 337140, 323231, 371205, 378746, 307175, 340299


#### as we can see the data is comma separated (also checked manually)

In [5]:
with open('expired_invoices.txt', 'r') as file:
    content = file.read()
    expired_inv = [int(id_str.strip()) for id_str in content.split(',')]

In [6]:
print(expired_inv)

[305869, 377307, 391273, 385290, 331902, 397723, 343695, 340601, 347510, 325156, 379387, 348894, 322229, 326649, 377960, 325063, 369378, 319405, 383681, 367288, 356552, 394428, 381476, 381457, 349879, 343254, 366751, 330931, 363263, 326452, 379687, 356532, 383235, 313012, 368913, 352442, 315960, 351096, 331193, 392657, 352391, 389528, 338547, 379961, 337140, 323231, 371205, 378746, 307175, 340299]


## DataExtractor class creation

In [7]:
# checking dtypes
df = pd.DataFrame(invoices)

print("Column Data Types:")
print(df.dtypes)

Column Data Types:
id            object
created_on    object
items         object
dtype: object


In [8]:
import pandas as pd
import pickle as pk
from datetime import datetime

class DataExtractor:
    def __init__(self, invoice_file, expired_file):
        self.invoice_file = invoice_file
        self.expired_file = expired_file
        self.data = None
        self.expired_invoices = None
    
    def load_data(self):
        try:
            with open(self.invoice_file, 'rb') as file:
                self.data = pk.load(file)
        except Exception as e:
            print(f"Error loading invoice file: {e}")
        
        try:
            with open(self.expired_file, 'r') as file:
                content = file.read()
                self.expired_invoices = [int(id_str.strip()) for id_str in content.split(',')]
        except Exception as e:
            print(f"Error loading expired invoices file: {e}")
    
    def flatten_items(self, invoice):
        try:
            invoice_id = int(invoice['id'])
            created_on = pd.to_datetime(invoice['created_on'], errors='coerce')
            if pd.isna(created_on):
                raise ValueError("Invalid date")
            invoice_total = sum(self.get_valid_price(item) for item in invoice['items'])
            
            flattened_data = []
            for item in invoice['items']:
                if not self.is_valid_item(item):
                    continue
                invoiceitem_id = item['item']['id']
                invoiceitem_name = item['item']['name']
                unit_price = item['item']['unit_price']
                item_type = self.map_type(item['item']['type'])
                quantity = item['quantity']
                total_price = unit_price * quantity
                percentage_in_invoice = total_price / invoice_total if invoice_total > 0 else 0
                is_expired = invoice_id in self.expired_invoices
                
                flattened_data.append({
                    'invoice_id': invoice_id,
                    'created_on': created_on,
                    'invoiceitem_id': invoiceitem_id,
                    'invoiceitem_name': invoiceitem_name,
                    'type': item_type,
                    'unit_price': unit_price,
                    'total_price': total_price,
                    'percentage_in_invoice': percentage_in_invoice,
                    'is_expired': is_expired
                })
            return flattened_data
        except Exception as e:
            print(f"Skipping processing invoice {invoice.get('id', 'unknown')}: {e}")
            return []

    @staticmethod
    def map_type(type_code):
        type_conversion = {0: 'Material', 1: 'Equipment', 2: 'Service', 3: 'Other'}
        return type_conversion.get(type_code, 'Unknown')
    
    @staticmethod
    def is_valid_item(item):
        try:
            _ = int(item['item']['id'])
            _ = str(item['item']['name'])
            _ = float(item['item']['unit_price'])
            _ = int(item['quantity'])
            _ = int(item['item']['type'])
            return True
        except ValueError:
            print(f"Invalid item data: {item}")
            return False

    @staticmethod
    def get_valid_price(item):
        try:
            return item['item']['unit_price'] * item['quantity']
        except:
            return 0
    
    def transform_data(self):
        all_flattened_data = []
        for invoice in self.data:
            flattened_data = self.flatten_items(invoice)
            all_flattened_data.extend(flattened_data)
        
        # Convert to DataFrame
        df = pd.DataFrame(all_flattened_data)
        
        # Ensure correct column types
        df['invoice_id'] = df['invoice_id'].astype(int)
        df['created_on'] = pd.to_datetime(df['created_on'])
        df['invoiceitem_id'] = df['invoiceitem_id'].astype(int)
        df['invoiceitem_name'] = df['invoiceitem_name'].astype(str)
        df['type'] = df['type'].astype(str)
        df['unit_price'] = df['unit_price'].astype(int)
        df['total_price'] = df['total_price'].astype(int)
        df['percentage_in_invoice'] = df['percentage_in_invoice'].astype(float)
        df['is_expired'] = df['is_expired'].astype(bool)
        
        # Sort the DataFrame
        df = df.sort_values(by=['invoice_id', 'invoiceitem_id']).reset_index(drop=True)
        
        return df
    
    def save_to_csv(self, df, filename):
        try:
            df.to_csv(filename, index=False)
            print(f"DataFrame successfully saved to {filename}")
        except Exception as e:
            print(f"Error saving DataFrame to CSV: {e}")
    
    def check_dtypes(self):
        df = self.transform_data()
        print(df.dtypes)

# Usage
extractor = DataExtractor('invoices_new.pkl', 'expired_invoices.txt')
extractor.load_data()
final_df = extractor.transform_data()
extractor.save_to_csv(final_df, 'final_invoices.csv')
extractor.check_dtypes()

Skipping processing invoice 365371O: invalid literal for int() with base 10: '365371O'
Skipping processing invoice 343695: unsupported operand type(s) for +: 'int' and 'str'
Skipping processing invoice 374089O: invalid literal for int() with base 10: '374089O'
Invalid item data: {'item': {'id': 195938, 'name': 'ii_195938', 'unit_price': 129, 'type': 'O'}, 'quantity': 2}
Invalid item data: {'item': {'id': 137032, 'name': 'ii_137032', 'unit_price': 116, 'type': 'O'}, 'quantity': 5}
Skipping processing invoice 397723O: invalid literal for int() with base 10: '397723O'
Skipping processing invoice 325156O: invalid literal for int() with base 10: '325156O'
Skipping processing invoice 326649O: invalid literal for int() with base 10: '326649O'
Skipping processing invoice 379961: 'items'
Skipping processing invoice 385290O: invalid literal for int() with base 10: '385290O'
Skipping processing invoice 381476O: invalid literal for int() with base 10: '381476O'
Skipping processing invoice 350506: 

In [10]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
